###### XGBOOST Parameter tuning
https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [104]:
#http://drivendata.co/blog/worldbank-poverty-benchmark/

%matplotlib inline

import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)

# data directory
DATA_DIR = os.path.join(r'C:\Users\piush\Desktop\Dataset\world_bank_poverty_data\household')

In [105]:
data_paths = {'A': {'train': os.path.join(DATA_DIR,  'A_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'A_hhold_test.csv')}, 
              
              'B': {'train': os.path.join(DATA_DIR,  'B_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'B_hhold_test.csv')}, 
              
              'C': {'train': os.path.join(DATA_DIR,  'C_hhold_train.csv'), 
                    'test':  os.path.join(DATA_DIR,  'C_hhold_test.csv')}}

In [106]:
# load training data
a_train = pd.read_csv(data_paths['A']['train'], index_col='id')
b_train = pd.read_csv(data_paths['B']['train'], index_col='id')
c_train = pd.read_csv(data_paths['C']['train'], index_col='id')

In [107]:
# load test data
a_test = pd.read_csv(data_paths['A']['test'], index_col='id')
b_test = pd.read_csv(data_paths['B']['test'], index_col='id')
c_test = pd.read_csv(data_paths['C']['test'], index_col='id')

In [108]:
a_train.shape

(8203, 345)

In [109]:
a_test.shape

(4041, 344)

In [110]:
df = pd.concat([a_train,a_test])

In [111]:
class LabelCount(object):

    def __init__(self, columns, new_column=False):
        self.count_dict = {}
        self.columns = columns
        self.new_column = new_column
        
    
    def fit(self, df):

        for column in self.columns:
            count = df[column].value_counts()
            
            self.count_dict[column] = count.to_dict()
        
    def transform(self, df):
        for column in self.columns:
            
            new_column_name = column
            
            if self.new_column:
                new_column_name = column + "_label_count"

            missing = 1
            df[new_column_name] = df[column].apply(lambda x : self.count_dict[column].get(x, missing))            


###### Label Count


In [112]:
for i in list(df.select_dtypes(include=['object']).columns.values):
    lc = LabelCount([i])
    lc.fit(df)
    lc.transform(df)
    

In [113]:
y = a_train['poor']

In [114]:
df = df.drop(['poor','country'], axis = 1)

In [115]:
train = df[:len(a_train)]

In [116]:
test = df[len(a_train):]

##### Standard Scalar processing

In [117]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
scaler = RobustScaler()
scaler.fit(df)

train_data_scaled = scaler.transform(train)
test_data_scaled = scaler.transform(test)

In [186]:
from sklearn import model_selection
from sklearn.metrics import log_loss

#from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

test_size = 0.33

seed = 7

X_train, X_test, Y_train, Y_test = model_selection.train_test_split(train_data_scaled, y, test_size=test_size, random_state=seed)

#model = LogisticRegression()
#model = RandomForestClassifier(n_estimators=10, random_state=0)
model = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05, objective= "binary:logistic",min_child_weight = 1,scale_pos_weight = 1)

model.fit(X_train, Y_train)

y_pred = model.predict_proba(X_test)

log_loss(Y_test, y_pred)

#print("f1_score: {}".format(f1_score(, average='micro')))

In [217]:
#k-fold 

num_instances = len(train_data_scaled)
seed = 7

kfold = model_selection.KFold(n_splits=5, random_state=seed)
#model = LogisticRegression()
model = XGBClassifier(max_depth=3, n_estimators=300, learning_rate=0.05, objective= "binary:logistic",min_child_weight = 1,scale_pos_weight = 1)


model_selection.cross_val_score(model, train_data_scaled, y, cv=kfold, scoring='neg_log_loss').mean()

-0.2925203492504474

0.2959906175178581

In [188]:
model.fit(train_data_scaled,y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=300, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [192]:
a_preds = model.predict_proba(test_data_scaled)

In [193]:
def make_country_sub(preds, test_feat, country):
    # make sure we code the country correctly
    country_codes = ['A', 'B', 'C']
    
    # get just the poor probabilities
    country_sub = pd.DataFrame(data=preds[:, 1],  # proba p=1
                               columns=['poor'], 
                               index=test_feat.index)

    
    # add the country code for joining later
    country_sub["country"] = country
    return country_sub[["country", "poor"]]


In [201]:
# convert preds to data frames
a_sub = make_country_sub(a_preds, a_test, 'A')

In [202]:
b_sub = pd.read_csv("b_sub.csv")

In [203]:
c_sub = pd.read_csv("c_sub.csv")

In [204]:
submission = pd.concat([a_sub, b_sub, c_sub])

In [205]:
submission = submission.drop("id", axis = 1)

In [206]:
submission.shape

(8832, 2)

In [207]:
submission.to_csv('submission_X_G_tr.csv')

In [209]:
submission.tail()

,country,poor
3182,C,0.000047
3183,C,0.000046
3184,C,0.000439
3185,C,0.000026
3186,C,0.000088
